# Merging SOTU and Newspaper Data

In [1]:
import os
import pandas as pd

In [2]:
newspapers_file = 'data/newspapers.json'
sotu_file = 'data/sotu.json'
output_file = 'data/merged.json'

## Preperation of News Media Files

In [3]:
dfn = pd.read_json(newspapers_file)
dfn.head()

,title,link,YEAR,PRESIDENT,newspaper,text
0,Magnet for Democrats Speculation Focuses on Go...,https://www.proquest.com/latimes/docview/29225...,1986,Reagan,LA Times,He is perhaps the best orator in the Democrati...
1,"Pumped Up for Event, Diver Finds Feat a B-r-r-...",https://www.proquest.com/latimes/docview/29224...,1986,Reagan,LA Times,-Brad Graske spent the day under the ice on Mc...
2,D-U-M-B!: [Home Edition],https://www.proquest.com/latimes/docview/29226...,1986,Reagan,LA Times,What is it about Reagan Administration secreta...
3,Ex-Black Panther Running for Senate Cleaver Ch...,https://www.proquest.com/latimes/docview/29224...,1986,Reagan,LA Times,The Los Angeles political consultant reached a...
4,First Daughter's Novel Reminiscent of Real Lif...,https://www.proquest.com/latimes/docview/29222...,1986,Reagan,LA Times,"This book is not, repeat not, the story of Ron..."


## Preperation of SOTU Files

In [4]:
dfs = pd.read_json(sotu_file)
dfs = dfs.rename(columns={'text': 'SOTU'})
dfs.head()

,link,date,president,SOTU
0,https://www.presidency.ucsb.edu/documents/addr...,2024-03-07,"Joseph R. Biden, Jr.","[Before speaking, the President presented his ..."
1,https://www.presidency.ucsb.edu/documents/addr...,2023-02-07,"Joseph R. Biden, Jr.","The President. Mr. Speaker——\n[At this point, ..."
2,https://www.presidency.ucsb.edu/documents/addr...,2022-03-01,"Joseph R. Biden, Jr.","The President. Thank you all very, very much. ..."
3,https://www.presidency.ucsb.edu/documents/addr...,2020-02-04,"Joseph R. Biden, Jr.",The President. Thank you very much. Thank you....
4,https://www.presidency.ucsb.edu/documents/addr...,2019-02-05,"Joseph R. Biden, Jr.","The President. Madam Speaker, Mr. Vice Preside..."


In [5]:
dfs['YEAR'] = dfs['date'].dt.year

## Merging files

In [6]:
df = pd.merge(
    left=dfn, left_on='YEAR',
    right=dfs[['SOTU', 'president', 'YEAR']], right_on='YEAR',
    how='left'
)
df.shape

(4993, 8)

In [7]:
df.isna().sum()

title           0
link            0
YEAR            0
PRESIDENT       0
newspaper       0
text            0
SOTU         1038
president    1038
dtype: int64

In [8]:
df['YEAR'].loc[df['SOTU'].isna()].unique()

array([1989, 1993, 2001, 2009, 2017, 2021])

In [28]:
df = df.loc[~df['SOTU'].isna()]
df.shape

(3955, 8)

In [29]:
df.head()

,title,link,YEAR,PRESIDENT,newspaper,text,SOTU,president
0,Magnet for Democrats Speculation Focuses on Go...,https://www.proquest.com/latimes/docview/29225...,1986,Reagan,LA Times,He is perhaps the best orator in the Democrati...,"Mr. Speaker, Mr. President, distinguished Memb...","Joseph R. Biden, Jr."
1,"Pumped Up for Event, Diver Finds Feat a B-r-r-...",https://www.proquest.com/latimes/docview/29224...,1986,Reagan,LA Times,-Brad Graske spent the day under the ice on Mc...,"Mr. Speaker, Mr. President, distinguished Memb...","Joseph R. Biden, Jr."
2,D-U-M-B!: [Home Edition],https://www.proquest.com/latimes/docview/29226...,1986,Reagan,LA Times,What is it about Reagan Administration secreta...,"Mr. Speaker, Mr. President, distinguished Memb...","Joseph R. Biden, Jr."
3,Ex-Black Panther Running for Senate Cleaver Ch...,https://www.proquest.com/latimes/docview/29224...,1986,Reagan,LA Times,The Los Angeles political consultant reached a...,"Mr. Speaker, Mr. President, distinguished Memb...","Joseph R. Biden, Jr."
4,First Daughter's Novel Reminiscent of Real Lif...,https://www.proquest.com/latimes/docview/29222...,1986,Reagan,LA Times,"This book is not, repeat not, the story of Ron...","Mr. Speaker, Mr. President, distinguished Memb...","Joseph R. Biden, Jr."


In [30]:
df.to_json(output_file, orient='records')